In [1]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
# !mkdir /home/aistudio/external-libraries
# !pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [2]:
# 程序在/work/code目录下， 加入路径
import sys 
sys.path.append('/home/aistudio/work/code')
import logging
logger = logging.getLogger(__name__)
# fly_paddle是唯一需要的模块
# fly_paddle is the only module users need to interact with
import fly_paddle as fp

In [3]:
# args 几乎包括所有需要的参数
# fp.parse_args2() 根据任务初始化args, 要用到的任务包括： ‘train', 'predict'
# args is a structure containing most (if not all) parameters
# fp.parse_args2() initializes args based on the task to run, such as "train", "predict"
args, _ = fp.parse_args2('train')
fp.misc.logging_config(logging, logfile=args.log, lineno=False, level=args.verbose) # 
print(fp.gwio.json_str(args.__dict__))

{
   "action": "train",
   "argv": "-h",
   "verbose": 1,
   "resume": false,
   "load_dir": null,
   "save_dir": null,
   "save_level": 2,
   "save_grpby": ["epoch", "batch"],
   "log": "fly_paddle-May12.log",
   "data_args": "======= data args =======",
   "data_dir": "data",
   "data_name": "predict",
   "data_suffix": ".pkl",
   "data_size": 0,
   "test_size": 0.1,
   "split_seed": null,
   "input_genre": "Seq",
   "input_fmt": "NLC",
   "seq_length": [0, 512, -1],
   "residue_fmt": "vector",
   "residue_nn": 0,
   "residue_dbn": false,
   "residue_attr": false,
   "residue_extra": false,
   "label_genre": "upp",
   "label_fmt": "NL",
   "label_tone": "none",
   "label_ntype": 2,
   "label_smooth": false,
   "net_args": "======= net args =======",
   "net_src_file": "/home/aistudio/work/code/paddle_nets.py",
   "net": "lazylinear",
   "resnet": false,
   "act_fn": "relu",
   "norm_fn": "none",
   "norm_axis": -1,
   "dropout": 0.2,
   "feature_dim": 1,
   "embed_dim": 32,
   "embed

In [4]:
# 注意： 根据不同的网络等等需要， args可能包含一些用不到的参数
# Attention: some parameters in args may not be used depending on the network etc.
# 两种更新args的方法： 1） args.update(**dict), 2) args.[key] = value
# Two main ways to update values in args: 1) args.update(**dict), 2) args.[key] = value
args.update(data_dir='work/data', data_name='train', residue_dbn=True, residue_extra=True)

# 网络参数 （net parameters): 
# 除非特意说明， 所有的隐藏层的维度为32. 
# 训练中发现高维度和深度的网络并不能给出更好的结果！
# The dimensions of all hidden layers are 32 unless noted otherwise
# Larger and deeper nets gave similar, but no better, performances!
args.net='seq2seq_attnlstmconv1d'  # the net name defined in paddle_nets.py
args.linear_num = 1 # the number of linear feedforward layers
args.attn_num = 1 # the number of transformer encoder layers
args.lstm_num = 1 # the number of bidirectional lstm layers
args.conv1d_num = 1 # the number of 1D convolutional layers
# 输出模块由三个线性层组成， 维度分别为32, 32, 2
# three linear layers for the final output, with dimensions of 32, 32, and 2, respectively
args.output_dim = [32, 32, 2] 
args.norm_fn = 'layer' # layer normalization
args.batch_size = 1 # 1 is used in consideration of the layer norm above
# 最后递交用的损失函数选为 softmax+bce, 也可以用 softmax+mse, 结果几乎一样
args.loss_fn = ['softmax+bce'] # softmax is needed here as the final output has a dimension of 2
args.label_tone = 'soft'
args.loss_sqrt = True # sqrt(loss) is only necessary for softmax+mse
args.loss_padding = False # exclude padded residues from loss
# 需要运行autoconfig_args()来消除参数的不一致性
# autoconfig_args() is needed to resolve inconsistencies between parameters
args = fp.autoconfig_args(args)


In [5]:
# 建立和检测模型 （Get and inspect the model）
model = fp.get_model(args)

2021-05-12 22:34:38,403 - INFO - Used net definition: /home/aistudio/work/code/paddle_nets.py
2021-05-12 22:34:38,487 - INFO - {'total_params': 36418, 'trainable_params': 36418}
2021-05-12 22:34:38,488 - INFO - Optimizer method: adam
2021-05-12 22:34:38,489 - INFO -    learning rate: 0.003
2021-05-12 22:34:38,489 - INFO -     lr_scheduler: reduced
2021-05-12 22:34:38,489 - INFO -     weight decay: none
2021-05-12 22:34:38,490 - INFO -          l1decay: 0.0001
2021-05-12 22:34:38,490 - INFO -          l2decay: 0.0001
2021-05-12 22:34:38,490 - INFO - Getting loss function: ['softmax+bce']


-------------------------------------------------------------------------------------------------------------------------------------
      Layer (type)                          Input Shape                                  Output Shape                   Param #    
   MyEmbeddingLayer-1                      [[2, 512, 10]]                                [2, 512, 10]                      0       
        Linear-1                           [[2, 512, 10]]                                [2, 512, 32]                     352      
         ReLU-1                            [[2, 512, 32]]                                [2, 512, 32]                      0       
       LayerNorm-1                         [[2, 512, 32]]                                [2, 512, 32]                     64       
        Dropout-1                          [[2, 512, 32]]                                [2, 512, 32]                      0       
     MyLinearTower-1                       [[2, 512, 10]]                 

In [6]:
# 读取数据 （read in data)
midata = fp.get_midata(args)
train_data, valid_data = fp.train_test_split(midata, test_size=0.1)

2021-05-12 22:34:38,496 - INFO - Loading data: work/data/train.pkl
2021-05-12 22:34:38,545 - INFO -    # of data: 5000,  max seqlen: 500, user seq_length: [0, 512, -1]
2021-05-12 22:34:38,546 - INFO -  residue fmt: vector, nn: 0, dbn: True, attr: False, genre: upp
2021-05-12 22:34:38,567 - INFO - Selected 5000 data sets with length range: [0, 512, -1]
2021-05-12 22:34:43,111 - INFO - Processing upp data...


In [7]:
# 训练模型， 最后的loss应该在[0.52, 0.53]区间内. 
# 每epoch需要五分钟左右(在CPU上)， 自然结束需要～20个epoch
train_loss, valid_loss = fp.train(model, train_data, num_epochs=21, validate_callback = fp.func_partial(fp.validate_in_train, midata=valid_data, save_dir='./'))

2021-05-12 22:34:43,410 - INFO - Training, data size: 4500
2021-05-12 22:34:43,412 - INFO -          batch size: 1
2021-05-12 22:34:43,412 - INFO -             shuffle: True
2021-05-12 22:34:43,413 - INFO -        # of batches: 4500
2021-05-12 22:34:43,414 - INFO -      recap interval: 151
2021-05-12 22:34:43,414 - INFO -   validate interval: 450
2021-05-12 22:34:43,609 - INFO -         # of epochs: 7
2021-05-12 22:34:43,610 - INFO -        loss padding: False
2021-05-12 22:34:44,661 - INFO - Epoch/batch: 0/   0, ibatch:    0, loss: 0.8624, std: 0.4525
2021-05-12 22:34:52,033 - INFO - loss: 0.8656, std: 0.4468
2021-05-12 22:34:57,985 - INFO - Epoch/batch: 0/ 151, ibatch:  151, loss: 0.6957, std: 0.6241
2021-05-12 22:35:03,903 - INFO - Epoch/batch: 0/ 302, ibatch:  302, loss: 0.6100, std: 0.6832
2021-05-12 22:35:19,032 - INFO - loss: 0.5681, std: 0.5923
2021-05-12 22:35:19,050 - INFO - Saved model states in: earlystop_0.5681
2021-05-12 22:35:19,051 - INFO - Saved net python code: earlys

Epoch 22: ReduceOnPlateau set learning rate to 0.0027.


2021-05-12 22:38:05,350 - INFO - Epoch/batch: 0/3322, ibatch: 3322, loss: 0.5692, std: 0.6381
2021-05-12 22:38:11,037 - INFO - Epoch/batch: 0/3473, ibatch: 3473, loss: 0.5547, std: 0.6376
2021-05-12 22:38:25,727 - INFO - loss: 0.5463, std: 0.6225
2021-05-12 22:38:26,682 - INFO - Epoch/batch: 0/3624, ibatch: 3624, loss: 0.5531, std: 0.6339
2021-05-12 22:38:32,134 - INFO - Epoch/batch: 0/3775, ibatch: 3775, loss: 0.5434, std: 0.6169
2021-05-12 22:38:38,093 - INFO - Epoch/batch: 0/3926, ibatch: 3926, loss: 0.5442, std: 0.6187
2021-05-12 22:38:52,639 - INFO - loss: 0.5446, std: 0.5813
2021-05-12 22:38:53,676 - INFO - Epoch/batch: 0/4077, ibatch: 4077, loss: 0.5489, std: 0.6292
2021-05-12 22:38:59,358 - INFO - Epoch/batch: 0/4228, ibatch: 4228, loss: 0.5147, std: 0.6061
2021-05-12 22:39:04,992 - INFO - Epoch/batch: 0/4379, ibatch: 4379, loss: 0.5401, std: 0.6170
2021-05-12 22:39:19,286 - INFO - loss: 0.5410, std: 0.5971
2021-05-12 22:39:19,301 - INFO - Saved model states in: earlystop_0.541

Epoch 40: ReduceOnPlateau set learning rate to 0.0024300000000000003.


2021-05-12 22:40:57,627 - INFO - Epoch/batch: 1/1510, ibatch: 6010, loss: 0.5491, std: 0.6243
2021-05-12 22:41:03,152 - INFO - Epoch/batch: 1/1661, ibatch: 6161, loss: 0.5363, std: 0.6134
2021-05-12 22:41:17,764 - INFO - loss: 0.5413, std: 0.5808
2021-05-12 22:41:18,297 - INFO - Epoch/batch: 1/1812, ibatch: 6312, loss: 0.5456, std: 0.6188
2021-05-12 22:41:24,083 - INFO - Epoch/batch: 1/1963, ibatch: 6463, loss: 0.5422, std: 0.6234
2021-05-12 22:41:29,918 - INFO - Epoch/batch: 1/2114, ibatch: 6614, loss: 0.5420, std: 0.6287
2021-05-12 22:41:44,636 - INFO - loss: 0.5363, std: 0.6298
2021-05-12 22:41:44,651 - INFO - Saved model states in: earlystop_0.5363
2021-05-12 22:41:44,653 - INFO - Saved net python code: earlystop_0.5363/paddle_nets.py
2021-05-12 22:41:44,659 - INFO - Saved best model: earlystop_0.5363
2021-05-12 22:41:44,660 - INFO - Removing earlystop model: earlystop_0.5385
2021-05-12 22:41:45,220 - INFO - Epoch/batch: 1/2265, ibatch: 6765, loss: 0.5191, std: 0.5976
2021-05-12 22

...

Epoch 194: ReduceOnPlateau set learning rate to 0.0006176733962839472.


2021-05-12 23:05:02,035 - INFO - Epoch/batch: 6/2114, ibatch: 29114, loss: 0.5168, std: 0.6056
2021-05-12 23:05:17,485 - INFO - loss: 0.5352, std: 0.6549
2021-05-12 23:05:18,220 - INFO - Epoch/batch: 6/2265, ibatch: 29265, loss: 0.5374, std: 0.6219
2021-05-12 23:05:24,624 - INFO - Epoch/batch: 6/2416, ibatch: 29416, loss: 0.5278, std: 0.6099
2021-05-12 23:05:30,786 - INFO - Epoch/batch: 6/2567, ibatch: 29567, loss: 0.5336, std: 0.6082
2021-05-12 23:05:45,355 - INFO - loss: 0.5297, std: 0.6230
2021-05-12 23:05:46,074 - INFO - Epoch/batch: 6/2718, ibatch: 29718, loss: 0.5316, std: 0.6118
2021-05-12 23:05:51,751 - INFO - Epoch/batch: 6/2869, ibatch: 29869, loss: 0.5101, std: 0.5933
2021-05-12 23:05:57,691 - INFO - Epoch/batch: 6/3020, ibatch: 30020, loss: 0.5585, std: 0.6389
2021-05-12 23:06:11,971 - INFO - loss: 0.5288, std: 0.6188
2021-05-12 23:06:12,821 - INFO - Epoch/batch: 6/3171, ibatch: 30171, loss: 0.5221, std: 0.5998
2021-05-12 23:06:18,593 - INFO - Epoch/batch: 6/3322, ibatch: 3

Epoch 205: ReduceOnPlateau set learning rate to 0.0005559060566555524.


2021-05-12 23:06:44,739 - INFO - Epoch/batch: 6/3775, ibatch: 30775, loss: 0.5217, std: 0.6005
2021-05-12 23:06:50,344 - INFO - Epoch/batch: 6/3926, ibatch: 30926, loss: 0.5377, std: 0.6181
2021-05-12 23:07:04,219 - INFO - loss: 0.5282, std: 0.6075
2021-05-12 23:07:04,235 - INFO - Saved model states in: earlystop_0.5282
2021-05-12 23:07:04,236 - INFO - Saved net python code: earlystop_0.5282/paddle_nets.py
2021-05-12 23:07:04,242 - INFO - Saved best model: earlystop_0.5282
2021-05-12 23:07:04,243 - INFO - Removing earlystop model: earlystop_0.5285
2021-05-12 23:07:05,204 - INFO - Epoch/batch: 6/4077, ibatch: 31077, loss: 0.5398, std: 0.6215
2021-05-12 23:07:10,938 - INFO - Epoch/batch: 6/4228, ibatch: 31228, loss: 0.5397, std: 0.6125
2021-05-12 23:07:16,603 - INFO - Epoch/batch: 6/4379, ibatch: 31379, loss: 0.5114, std: 0.6015
2021-05-12 23:07:30,435 - INFO - loss: 0.5301, std: 0.6314
2021-05-12 23:07:31,908 - INFO - Epoch 6 average training loss: 0.5323 std: 0.6114
2021-05-12 23:07:31

  [500 rows x 7 columns]])

In [21]:
# 读取最后一个checkpoint目录 (忽略优化器state_dict读取的问题)
# read the last saved earlystop folder （ignore the error in optimizer state_dict loading)
fp.state_dict_load(model, model.validate_hist.saved_dirs[-1])
# 可以改动损失函数，检测mse损失
args.loss_fn = ['softmax+mse']
model.loss_fn = fp.get_loss_fn(args)
valid_loss = fp.validate(model, valid_data, verbose=1, batch_size=64) # try a larger batch_size, should make no difference though

2021-05-12 23:32:46,449 - INFO - Loading model states from: earlystop_0.5282
2021-05-12 23:32:46,454 - INFO - Loaded net state: earlystop_0.5282/net.state
2021-05-12 23:32:46,457 - WARNING - Error in optim state_dict loading!
2021-05-12 23:32:46,458 - INFO - Getting loss function: ['softmax+mse']
2021-05-12 23:32:46,459 - INFO - Validating, data size: 500
2021-05-12 23:32:46,460 - INFO -            batch size: 16
2021-05-12 23:32:46,460 - INFO -               shuffle: False
2021-05-12 23:32:46,461 - INFO -          # of batches: 32
2021-05-12 23:32:46,461 - INFO -        recap interval: 2
2021-05-12 23:32:46,461 - INFO -          loss padding: False
2021-05-12 23:32:48,681 - INFO - ibatch:    0, loss: 0.1712, std: 0.2547
2021-05-12 23:32:49,071 - INFO - ibatch:    2, loss: 0.2024, std: 0.2778
2021-05-12 23:32:49,473 - INFO - ibatch:    4, loss: 0.2365, std: 0.3232
2021-05-12 23:32:49,855 - INFO - ibatch:    6, loss: 0.1969, std: 0.2814
2021-05-12 23:32:50,237 - INFO - ibatch:    8, los

2021-05-12 23:12:51,048 - INFO - Getting loss function: ['softmax+mse']
2021-05-12 23:12:51,050 - INFO - Validating, data size: 500
2021-05-12 23:12:51,051 - INFO -            batch size: 128
2021-05-12 23:12:51,107 - INFO -               shuffle: False
2021-05-12 23:12:51,108 - INFO -          # of batches: 4
2021-05-12 23:12:51,109 - INFO -        recap interval: 1
2021-05-12 23:12:51,109 - INFO -          loss padding: False
2021-05-12 23:12:55,531 - INFO - ibatch:    0, loss: 0.2093, std: 0.2943
2021-05-12 23:12:56,960 - INFO - ibatch:    1, loss: 0.2082, std: 0.2944
2021-05-12 23:12:58,366 - INFO - ibatch:    2, loss: 0.2145, std: 0.3070
2021-05-12 23:12:59,645 - INFO - ibatch:    3, loss: 0.2212, std: 0.3156
2021-05-12 23:13:00,701 - INFO - Validate mean: 0.2131, std: 0.1073


In [18]:
# 读取预测数据， 存储预测结果
# Read in prediction data, and save the predicted results
predict_data = fp.get_midata(args, data_name='predict', seq_length=-1)
y_model, std_model = fp.predict(model, predict_data, save_dir='predict.files', batch_size=1)

2021-05-12 23:29:07,576 - INFO - Loading data: work/data/predict.pkl
2021-05-12 23:29:07,579 - INFO -    # of data: 112,  max seqlen: 861, user seq_length: -1
2021-05-12 23:29:07,608 - INFO -  residue fmt: vector, nn: 0, dbn: True, attr: False, genre: upp
2021-05-12 23:29:10,735 - INFO - Predicting, data size: 112
2021-05-12 23:29:10,738 - INFO -            batch size: 1
2021-05-12 23:29:10,739 - INFO -               shuffle: False
2021-05-12 23:29:10,739 - INFO -          # of batches: 112
2021-05-12 23:29:10,740 - INFO -        recap interval: 4
2021-05-12 23:29:10,740 - INFO - Predicted files will be saved in: predict.files
100%|██████████| 112/112 [00:07<00:00, 14.20it/s]
2021-05-12 23:29:18,639 - INFO - Completed prediction of 112 samples
